# Model assembly

## Modell Auswahl

Neuronales Netz, but WHY?????

## Modellimplementierung

In [1]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
class LolProDataset(Dataset):
    def __init__(self, data_file):
        data = pd.read_csv(data_file)
        self.labels = torch.tensor(data.pop('Win Rate').to_numpy(), device=device)
        self.features = torch.tensor(data.to_numpy(), device=device)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        return (self.features[idx], self.labels[idx])

    def split(self, test_rate):
        testc = int(self.__len__()*test_rate)
        trainc = int(self.__len__() - testc)
        return [trainc, testc]

In [4]:
class LolProNetwork(nn.Module):
    def __init__(self, network_stack):
        super(LolProNetwork, self).__init__()
        self.network_stack = network_stack

    def forward(self, x):
        logits = self.network_stack(x)
        return logits

In [15]:
lr = 0.0001
batch_size = 1
epochs = 100

network_stack = nn.Sequential(
    nn.Linear(20, 128),
    nn.Sigmoid(),
    nn.Linear(128, 64),
    nn.Sigmoid(),
    nn.Linear(64, 32),
    nn.Sigmoid(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

model = LolProNetwork(network_stack).to(device).double()

loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        test_loss, err = 0, 0

        with torch.no_grad():
            for X, y in dataloader:
                pred = model(X)
                test_loss += loss_fn(pred, y.unsqueeze(1)).item()
                err += torch.abs(pred - y.unsqueeze(1)).sum().data
        test_loss /= size
        err /= size
        print(f"Test Error: \n MAE: {(err):>8f}, Avg loss: {test_loss:>8f} \n")

In [17]:
dataset = LolProDataset('cleanDataTop+MidS10Final.csv')
train_data, test_data = random_split(dataset, dataset.split(0.1), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [18]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn, optimizer)

h 18
-------------------------------
loss: 0.162490  [    0/  500]
loss: 0.080401  [  100/  500]
loss: 0.038230  [  200/  500]
loss: 0.041508  [  300/  500]
loss: 0.031028  [  400/  500]
Test Error: 
 MAE: 0.107763, Avg loss: 0.107763 

Epoch 19
-------------------------------
loss: 0.223830  [    0/  500]
loss: 0.205742  [  100/  500]
loss: 0.051936  [  200/  500]
loss: 0.296225  [  300/  500]
loss: 0.019244  [  400/  500]
Test Error: 
 MAE: 0.108415, Avg loss: 0.108415 

Epoch 20
-------------------------------
loss: 0.107615  [    0/  500]
loss: 0.072413  [  100/  500]
loss: 0.083707  [  200/  500]
loss: 0.049833  [  300/  500]
loss: 0.211436  [  400/  500]
Test Error: 
 MAE: 0.105563, Avg loss: 0.105563 

Epoch 21
-------------------------------
loss: 0.063651  [    0/  500]
loss: 0.006262  [  100/  500]
loss: 0.038567  [  200/  500]
loss: 0.043569  [  300/  500]
loss: 0.360763  [  400/  500]
Test Error: 
 MAE: 0.106946, Avg loss: 0.106946 

Epoch 22
-------------------------------

In [27]:
my_test = [100, 5, 4, 1.5, 3.5, 9, 450, 0.5, 0.2, 600, 1.3, 0.6, 0.2, 0.3, 500, 30, 400, 0.3, 0.05, 40]
my_test_tensor = torch.tensor(my_test, device=device).double()
model(my_test_tensor)

tensor([0.5979], device='cuda:0', dtype=torch.float64,
       grad_fn=<SigmoidBackward>)